# Julia is fast

Very often, benchmarks are used to compare languages. These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences. These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself. One can read the notebook and see what happened on the author's Macbook Pro with a 4-core Intel Core I7, or run the notebook yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb.)

## Outline of this notebook
1. Define the sum function
2. Implementations & benchmarking of sum in...
    - C (hand-written)
    - C (hand-written with -ffast-math)
    - python (built-in)
    - python (numpy)
    - python (hand-written)
    - Julia (built-in)
    - Julia (hand-written)
    - Julia (hand-written with SIMD)
3. Summary of benchmarks

---------

## ``sum``: An easy enough function to understand
Consider the sum function ``sum(a)``, which computes

\begin{align}
\text{sum}(a) = \sum_{i=1}^{n} a_i
\end{align}

where $n$ is the length of ``a``.

In [1]:
a = rand(10^7) # 1D vector of random numbers, uniform on [0,1)

10000000-element Vector{Float64}:
 0.8480230254780928
 0.7494819452021777
 0.9871116429048478
 0.02713219249974297
 0.3089717648531326
 0.00031268753634361435
 0.17935912519018504
 0.8097015461946862
 0.5865204552607233
 0.6465751470261549
 0.66289009313373
 0.09943835033173909
 0.3367208520900331
 ⋮
 0.6983292789701314
 0.6510544565777115
 0.9590308417862617
 0.5442499294052464
 0.07425524191109356
 0.38117695374183136
 0.7851617024914284
 0.26688096960805696
 0.16552942929494896
 0.2129604917088963
 0.9870863450731623
 0.16115524216476074

In [2]:
sum(a)

5.000021249925878e6

The expected result is $0.5 \times 10^7$, since the mean of each entry is $0.5$

## Benchmarking a few ways in a few languages